In [ ]:

import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import random
from torchvision import models
from torchsummary import summary

from collections import Counter
import sys
import os
import pickle
import shutil


get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)
import time
import os

np.set_printoptions(precision=5)
'''angeleht an: https://nextjournal.com/gkoehler/pytorch-mnist
https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118'''

'angeleht an: https://nextjournal.com/gkoehler/pytorch-mnist\nhttps://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''Set your file directorys'''

googledrive_path="/content/drive/MyDrive/Colab Notebooks/MNist/"
local_path="C:/Users/Flo/Documents/Uni/Masterarbeit/Hanabi/Mnist handwritten digits/"

'''set global seeds'''

seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

'''Logging to screen options'''
print_distribution_data=False

'''number of training epochs for Network2'''
n_training_epochs = 1 #number of training epochs for every NN2.  
n_offsp_epoch = 10 #number of training and testing runs combined, to get an average for the performance of the Convu net.
n_testing_epochs = 3 # number of testing runs, per n_offsp_epoch

'''parameter Network2'''
n_samples = 100/n_training_epochs  #Number of training samples for NN2, distribution of data for 600: [68, 59, 66, 67, 47, 46, 65, 71, 53, 58])
n_test=500 #Number of test samples for NN2. Needs to be multiples of batch_size_test=500 )
batch_size_train = 20 # for precise n_samples number must be: n_samples%batch_size_train=0
batch_size_test = 500 #for precise n_test number must be: n_test%batch_size_test=0
learning_rate = 0.1
momentum = 0.5
log_interval = 10



'''Standard deviation for gaussian noise in Network1'''
'''!!! Important hyperparameter, >=1 gives bad results'''
std_modifier=0.05


# !!! ATM Overriden in main code part
'''number of offsprings per metaepoch'''
n_offsprings=25
'''number of metaopochs'''
n_metaepochs=3
''''Elitist: Keep parent in offspring_list as [0]:'''
elitist=True
#'''True keeps standard Convu weights from pytorch, leads to very good results from the beginning (cheating?)'''
#firstelitist=False'

'''06.02.2022 manually reduced to one layer'''
#'''NeuralNet'''
NNin1=1536
NNout1=10
#NNout2=10

'''Convunet'''
Convu_in1=1
Convu_out1=12
Convu_out2=24
kernelsize_=3 #if changed, NNin1 needs to be adapted aswell

'''Not sure about purpose of these variables anymore'''
#torch.use_deterministic_algorithms(False)
#torch.backends.cudnn.enabled = True
#torch.backends.cudnn.deterministic = False
#torch.backends.cudnn.benchmark = True


'Not sure about purpose of these variables anymore'

In [ ]:
torch.manual_seed(seed)
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=True, download=True,
                                            transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                            batch_size=batch_size_train, shuffle=True,num_workers=0)
torch.manual_seed(seed)
test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=False, download=True,
                                          transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                          batch_size=batch_size_test, shuffle=True,num_workers=0)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



In [ ]:

def test():
    neuralnetwork.eval()
    test_loss = 0
    correct = 0
    count=0
    test_targets = []
    '''Increases randomness (?)'''
    next(iter(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            
            '''reduce number of trainings samples'''
            if count >= n_test/batch_size_test:
                '''return distribution of data of test samples'''
                if print_distribution_data:
                    test_targets = torch.cat(test_targets)
                    string_distribution_data=test_targets.unique(return_counts=True)
                    print(string_distribution_data)
                    print("Number of testsamples:{:.2f}".format(sum(list(string_distribution_data[1]))))
                break
                
            test_targets.append(target)
            count+=1
            
            output = convunet(data)
            output = neuralnetwork(output)
            loss_func = nn.CrossEntropyLoss()   
        
            test_loss += loss_func(output, target)

            #test_loss += F.nll_loss(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            test_loss /= n_test
            test_losses.append(test_loss)
            #string=('\t\tTest set: Avg. loss: {:.4f}, Accuracy: {:.2f}/{:.2f} ({:.0f}%)\n'.format(test_loss, correct, n_test,100. * correct / n_test))
    '''Increases randomness (?)'''
    next(iter(test_loader))
    return test_loss, correct / n_test

In [ ]:
def train(epoch):
   
    neuralnetwork.train()
    
    train_targets = []
    count=0
    '''Increases randomness ?'''
    next(iter(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        
        '''reduce number of trainings samples'''
        if count >= n_samples/batch_size_train:
            '''return distribution of data of test samples'''
            if print_distribution_data:
                train_targets = torch.cat(train_targets)
                string_distribution_data=train_targets.unique(return_counts=True)
                print(string_distribution_data)
                print("Number of training samples:{:.2f}".format(sum(list(string_distribution_data[1]))))
            break
            
        train_targets.append(target)
        count+=1
        
        '''disable gradient methods for convu'''
        with torch.no_grad():
            output = convunet(data)
        output = neuralnetwork(output)   
        
        loss_func = nn.CrossEntropyLoss()   
        
        loss = loss_func(output, target)
        
        
        # clear gradients for training step
        optimizer.zero_grad()
        #print(loss)
        loss.backward()
        
        optimizer.step()
        
        
       
        next(iter(train_loader))
        if batch_idx % log_interval == 0:
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [ ]:
'''logging to screen variables'''
print_offsprings=True
print_distribution_data=False
use_sigma_decay=True #otherwise using constant sigma from config tab
sigma_start=1 
sigma_goal=0.05 #sigma goal after n_metaepochs
        
'''choose either method, softmax or elitist=keep only best offspring'''
use_softmax=True
temperature=0.05
use_elitist=False


n_metaepochs=30 #overwriting variable from config tab, delete later
n_offsprings=100 #overwriting variable from config tab, delete later

In [ ]:


'''number of training epochs for Network2'''
n_offsp_epoch = 10
n_testing_epochs = 3

In [ ]:
mean_results=[]
std_results=[]
best_performer=[]
result_list_final=[]

pathandstuff()
log_variables()

convunet = Convu()

for m in range (n_metaepochs):

    '''sigma decay function over n_metaepochs, specify variables'''
    if use_sigma_decay:
        sigma_base=sigma_decay(sigma_start, sigma_goal, n_metaepochs)
        std_modifier=sigma_start*sigma_base**m

    metaseed=0+m #change seed and therefore data input for every metaepoch
    torch.manual_seed(seed)
    start_meta = time.time()
    logg("Metaepoch:{}".format(m))

    if m ==0:
        father_state_dic=deepcopy(convunet.state_dict())
        offspring_list=create_offsprings(n_offsprings, father_state_dic)
    if m >0:
        if use_softmax==True:
            results=np.array([float(x[2]) for x in result_list_final[m-1]])
            softmax_vec = softmax(results, temperature)
            print("Softmax_vec:", softmax_vec)
            w_offspring_list=weight_offspring_list(n_offsprings,offspring_list,softmax_vec)
            winner=dict(deepcopy(sum_list_of_dics(w_offspring_list)))
            offspring_list=create_offsprings(n_offsprings, winner)
            
        elif use_elitist==True:
            bestindex=best_index_bestperformer(result_list_final[m-1])
            father_state_dic=deepcopy(offspring_list[bestindex])
            offspring_list=create_offsprings(n_offsprings, father_state_dic, True)
        

    result_list_metaepoch=[]
    best_accur=0
    #create_log()
    #logg("Metaepoch:{}".format(n_metaepochs))

    for offsp_count in range (0, n_offsprings):
        start_offs = time.time()
        offsp_epoch_list=[]
        for offsp_epoch in range (n_offsp_epoch):
          
            '''#run every Offspring Epoch with different data shuffle, 
            but every offspring runs with the same shuffle! 
            Better comparison of results. 
            Differences are only accounted by difference in the Convu layer'''
            seed=metaseed+100+offsp_epoch 
            
            '''reset variables and NN for every run'''
            torch.manual_seed(seed) #Very important line to keep weights of NN2 constant
            neuralnetwork = NN()

            optimizer = torch.optim.Adam(neuralnetwork.parameters(), lr=learning_rate) #better performance
            #optimizer = optim.SGD(neuralnetwork.parameters(), lr=learning_rate, momentum=momentum)
            train_losses = []
            train_counter = []
            test_losses = []
            test_counter = [i*len(train_loader.dataset) for i in range(n_training_epochs + 1)]

            '''load offspring into convu'''
            #convunet.load_state_dict(best_convu)
            convunet.load_state_dict(offspring_list[offsp_count])

            '''run n_training_epochs of training for the same NN2'''
            for tr_epoch in range(n_training_epochs):
                train(tr_epoch)

            '''run multiple test runs for each Offspring Epoch for more accurate test results'''
            test_epoch_results=[]
            for te_epoch in range(1, n_testing_epochs +1):
                te_test_loss, te_accuracy = test()
                test_epoch_results.append([te_test_loss,te_accuracy])

            '''save offspring epochs results'''
            test_loss=np.mean(np.array([float(x[0]) for x in test_epoch_results]))
            accuracy=np.mean(np.array([float(x[1]) for x in test_epoch_results]))
            std=np.std(np.array([float(x[1]) for x in test_epoch_results]))
            offsp_epoch_list.append([test_loss,accuracy,std])
        
        '''Results for every offspring (average over n_offsp_epoch = 10 and in every n_offsp_epoch: n_testing_epochs = 3 '''
        test_loss_offsp=np.mean(np.array([float(x[0]) for x in offsp_epoch_list]))
        accuracy_offsp=np.mean(np.array([float(x[1]) for x in offsp_epoch_list]))
        std_offsp=np.std(np.array([float(x[1]) for x in offsp_epoch_list]))
        result_list_metaepoch.append([(m,offsp_count), test_loss_offsp, accuracy_offsp, std_offsp])
        
        '''save best performer in variable for further testing'''
        if accuracy_offsp>best_accur:
            best_convu=offspring_list[offsp_count]
            best_accur=accuracy_offsp

        '''log to screen'''
        if print_offsprings:
            np.set_printoptions(precision=3)
            logg("\tOffspring {}".format(offsp_count))
            logg("\tAccuracy of different offspring epochs:",np.array([float(x[1]) for x in offsp_epoch_list]))
            logg("\tAverage accuracy for Offspring: {:.2f}, std: {:.2f}".format(accuracy_offsp,std_offsp))
            logg("\tTime per offspring:{:.1f} s\n".format(time.time() - start_offs))


    
    '''Stats Metaepoch'''
    metaepoch_mean=np.mean(np.array([float(x[2]) for x in result_list_metaepoch]))
    metaepoch_std=np.std(np.array([float(x[2]) for x in result_list_metaepoch]))
    metaepoch_idx_best=np.where(result_list_metaepoch==np.max([float(x[2]) for x in  result_list_metaepoch]))[0][0]
    metaepoch_best_offspring_mean=[float(x[2]) for x in result_list_metaepoch][metaepoch_idx_best]
    metaepoch_best_offspring_std=[float(x[3]) for x in result_list_metaepoch][metaepoch_idx_best]

    result_list_final.append(result_list_metaepoch)
    best_performer.append([metaepoch_best_offspring_mean,metaepoch_best_offspring_std])

    logg("Metaepoch:{}".format(m))
    logg("Best Offspring:{}, with {:.2f} accuracy and {:.2f} std".format(metaepoch_idx_best,metaepoch_best_offspring_mean,metaepoch_best_offspring_std))
    logg("Time per metaepoch:{:.1f}s".format(time.time() - start_meta))
    logg("accuracy_list metaepoch {}:".format(m), np.array([float(x[2]) for x in result_list_metaepoch]))
    logg("average mean metaepoch:{:.2f}".format(metaepoch_mean))
    logg("average std metaepoch:{:.2f}".format(metaepoch_std))
    logg("\n")
    
    logg("Development best performer:", np.array(best_performer))
    save_p(best_convu, "best_convu",save_path) #save weights of best convu to pickle
    print(plot_mean([x[0] for x in best_performer],[x[1] for x in best_performer]))



on google
Log path: /content/drive/MyDrive/Colab Notebooks/MNist/Logs/03.05.2022_16.02
Metaepoch:0
	Offspring 0
	Accuracy of different offspring epochs: [0.119 0.097 0.163 0.079 0.113 0.093 0.169 0.099 0.198 0.203]
	Average accuracy for Offspring: 0.13, std: 0.04
	Time per offspring:11.6 s

	Offspring 1
	Accuracy of different offspring epochs: [0.112 0.097 0.223 0.079 0.233 0.097 0.188 0.117 0.09  0.177]
	Average accuracy for Offspring: 0.14, std: 0.06
	Time per offspring:11.6 s

	Offspring 2
	Accuracy of different offspring epochs: [0.252 0.174 0.116 0.203 0.223 0.291 0.147 0.082 0.09  0.211]
	Average accuracy for Offspring: 0.18, std: 0.07
	Time per offspring:11.7 s

	Offspring 3
	Accuracy of different offspring epochs: [0.217 0.095 0.108 0.151 0.205 0.109 0.115 0.093 0.103 0.2  ]
	Average accuracy for Offspring: 0.14, std: 0.05
	Time per offspring:12.2 s

	Offspring 4
	Accuracy of different offspring epochs: [0.154 0.217 0.181 0.293 0.284 0.11  0.105 0.191 0.12  0.184]
	Average accu

TypeError: ignored

In [ ]:
def softmax(results,temp):
    x = [z/temp for z in results]
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def save_p(obj, name,direc):
    path=os.path.join(direc,name)
    with open(path+".pkl", 'wb') as f:

        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        f.close()


In [ ]:
'''logging to txt and print'''
def logg (string_, array=None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    print(string_)
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    print(string_, array)
  

In [ ]:
def pathandstuff():

    global save_txt
    global base_path
    global save_path

    if os.path.exists(local_path):
        '''Save running code file to log folder'''
        #nb_full_path = os.path.join(os.getcwd(), nb_name) #path of current notebook
        #shutil.copy2(nb_full_path, save_path) #save running code file to log folder
        print("on local")
        base_path=local_path
    elif os.path.exists(googledrive_path):
        print("on google")
        base_path=googledrive_path
    else:
        raise ValueError('Please specify save path or connect to Google Drive')
        
    logs_path=base_path+"Logs/"
    '''Set logging and temp paths'''
    timestamp=time.strftime("%d.%m.%Y_%H.%M")
    foldername=timestamp
    save_path=os.path.join(logs_path,foldername)
    print("Log path:",save_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    save_txt = os.path.join(save_path, 'Log_MNist_{}.txt'.format(foldername))
    
   
    

In [ ]:
def plot_mean(mean_results,std_results): 
    mean=mean_results
    std=std_results
    x_axis=np.arange(0,len(mean), 1)

    plt.xticks(range(0,len(mean)))

    mean = np.array(mean)
    std = np.array(std)

    lower_bound = mean - std
    upper_bound = mean + std

    plt.fill_between(x_axis, lower_bound, upper_bound, alpha=.3)
    plt.plot(mean,marker="o")
    plt.savefig(save_path+'/plot.png')
    return plt

In [ ]:
def sigma_decay(start, end, n_iter):
  return(end/start)**(1/n_iter)

In [ ]:
def index_nhighest_list(n,list):
  return [i for x, i in heapq.nlargest(n,((x, i) for i, x in enumerate(list)))]

In [ ]:
def matrix_distance(offspring_list):
    length=len(offspring_list)

    matrix=np.zeros((length, length))
    for c in range(length):
        for r in range(length):



            hilf=substract_two_dics(offspring_list[c],offspring_list[r])

            hilf2=square_dic(dict(hilf))
            sum_=dic_totalsum(hilf2)
            matrix[r][c]=sum_
    return matrix
    

In [ ]:
def substract_two_dics(a,b):
    a = Counter(a)
    a.subtract(b)
    return a

In [ ]:
'''logging to txt and print'''
def logg_to_file (string_, array=None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    
  

In [ ]:
def log_variables():
    logg_to_file(("seed = {}".format(seed)))
    logg_to_file (("n_training_epochs = {}".format(n_training_epochs)))
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    logg_to_file (("n_testing_epochs = {}".format(n_testing_epochs)))
    logg_to_file (("n_samples = {}".format(n_samples)))
    logg_to_file (("n_test = {}".format(n_test)))
    logg_to_file (("batch_size_train = {}".format(batch_size_train)))
    logg_to_file (("learning_rate = {}".format(learning_rate)))

    logg_to_file (("std_modifier = {}".format(std_modifier)))
    logg_to_file (("use_sigma_decay = {}".format(use_sigma_decay)))
    logg_to_file (("sigma_start = {}".format(sigma_start)))
    logg_to_file (("sigma_goal = {}".format(sigma_goal)))
    logg_to_file (("elitist = {}".format(elitist)))
    logg_to_file (("NNin1 = {}".format(NNin1)))
    logg_to_file (("NNout1 = {}".format(NNout1)))
    logg_to_file (("Convu_in1 = {}".format(Convu_in1)))
    logg_to_file (("Convu_out1 = {}".format(Convu_out1)))
    logg_to_file (("Convu_out2 = {}".format(Convu_out2)))

    logg_to_file (("kernelsize_ = {}".format(kernelsize_)))
    logg_to_file (("use_elitist = {}".format(use_elitist)))
    logg_to_file (("n_metaepochs = {}".format(n_metaepochs)))
    logg_to_file (("n_testing_epochs = {}".format(n_testing_epochs)))
                  
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    logg_to_file (("n_offsprings = {}".format(n_offsprings)))

    logg_to_file (("use_softmax = {}".format(use_softmax)))
    logg_to_file (("temperature = {}".format(temperature)))

                  
 

In [ ]:
def square_dic(my_dict):
    my_dict.update((x, y**2) for x, y in my_dict.items())
    return my_dict

In [ ]:
def dic_totalsum(my_dict):
    return float(torch.sum(sum(sum(x) for x in my_dict.values())))



In [ ]:
'''weighting every element in dic list with softmax weights from m'''
def weight_offspring_list(n_offsprings,offspring_list,m):
    for n in range(n_offsprings):

        for key in offspring_list[n]:    
            offspring_list[n][key] *=  m[n]
    return offspring_list

In [ ]:
'''needed for softmax'''
def sum_list_of_dics(dic_list):
    c = Counter()
    for d in w_offspring_list:
        c.update(d)
    return c

In [ ]:
'''get best performer from final resultlist[[0,2,0.8],[1,2,0.5],[2,2,0.2]'''
def best_index_bestperformer(lista):
    hilf=0
    index=None
    for i in range (len(lista)):
        if hilf<lista[i][2].item():
            index=i
            hilf=lista[i][2].item()
    return index


In [ ]:
'''creates list of statedic offsprings'''
def create_offsprings(n_offspr, fathersd,elitist=False):
    statedic_list=[]
    
    for i in range(0,n_offspr):
        hilfdic={}
        
        for param_tensor in fathersd:
            '''get size of convu layer '''
            size=fathersd[param_tensor].size()
            
            '''create vector with gaussian noise'''
            x = torch.zeros(size, dtype=torch.float64)
            x = x + torch.randn(size)* std_modifier
            
            '''add gaussian noise to convu weights'''
            hilfdic[param_tensor]=fathersd[param_tensor]+x
        
        statedic_list.append(hilfdic)
        
        '''Elitist feature: Keep the parent in the next offspring generation'''
        if elitist==True:
          statedic_list[0]=fathersd
    return statedic_list

In [ ]:
class Convu(nn.Module):
    def __init__(self):
        super(Convu, self).__init__() 
        
        self.requires_grad_(False)
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=Convu_in1,              
                out_channels=Convu_out1,            
                kernel_size=kernelsize_,              
                stride=1,                   
                padding=2),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(         
            nn.Conv2d(Convu_out1, Convu_out2, kernelsize_, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),)    
        
          
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        self.requires_grad_(False)
        return x    # return x for visualization

In [ ]:
'''single layer NN'''
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        
        self.fc1 = nn.Linear(NNin1, NNout1)
        

    def forward(self, x):
        
      
        x = self.fc1(x)
        return x